In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

## LLMChain

In [ ]:
from langchain.chains import LLMChain

In [ ]:
llm = ChatOpenAI(temperature=0)

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "Suggest movies of the following genre: {genre}"
)

In [ ]:
chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
genre = "comedy"
print(chain.run(genre))

## SimpleSequentialChain

In [ ]:
from langchain.chains import SimpleSequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Suggest movies of the following genre: {genre}"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [ ]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words review for the following \
    movies:{movie_names}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [ ]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [ ]:
genre = 'comedy'
print(overall_simple_chain.run(genre))

## SequentialChain

In [ ]:
from langchain.chains import SequentialChain

In [ ]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1: get movie names from genre
first_prompt = ChatPromptTemplate.from_template(
    "Suggest 2 movies of the following genre: {genre}"
)
# chain 1: input= genre and output= movie_names
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="movie_names"
                    )


In [ ]:
# prompt template 2: generating cast of movies
second_prompt = ChatPromptTemplate.from_template(
    "Give cast of the following movies: {movie_names}"
)
# chain 2: input= movie_names and output= cast
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="cast"
                    )

In [ ]:
# prompt template 3: generating reviews of movies
third_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words review for the following \
    movies:{movie_names}"
)
# chain 3: input= movie_names and output= reviews
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="reviews"
                      )

In [ ]:
# prompt template 4: counting actors and actresses 
fourth_prompt = ChatPromptTemplate.from_template(
    "Give count of actors and actresses in each movie from its cast list: {cast}"
)
# chain 4: input= cast, language and output= cast_count
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="cast_count"
                     )

In [ ]:
# overall_chain: input= genre 
# and output= cast, reviews, cast_count
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["genre"],
    output_variables=["cast", "reviews","cast_count"],
    verbose=True
)

In [ ]:
genre = 'comedy'
print(overall_chain(genre))

## RouterChain

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [ ]:
llm = ChatOpenAI(temperature=0)

In [ ]:
pre_purchase_template = """You are a very smart salesman. \
You have only one goal that is to try and get customers to buy products from you.\
Whenever customer asks about any product, you describe the product in such a way that they cant resist.\
Even when customer is confused ot dose not know what to buy, you describe about your products do that it sells.\
All this but at the same time dont let the customer know you are talking as a salesman.\

Here is a question that customer is asking:\
{input}"""

post_purchase_template = """You are handling cutomers who have post-purchase queries.\
You will keep the answers short, to the point and crsip so that customer feels satisfed.\
You may also try and retain the customers so they place repeat orders.\
You can give them some offer as well if you think that will help in customer to re-order.\

Here is a question that customer is asking:\
{input}"""

angry_customer_template = """You are dealing with an angry customer for some reason.\
You have to give a very poliet and sweet reply back and also be apologetic if needed.\
You will try and calm the customer and assure them about resolution.\

Here is a question that customer is asking:\
{input}"""

default_template = """ You are a customer service agent at an e-commerce.\
You handle any random questions that user may ask which is not realted to our business.\
You may answer the user in a very short manner or you if you think you can deny answering as well\
as we dont want to entertain much of the customer queries which are unrelated.\

Here is a question that customer is asking:\
{input}"""

In [ ]:
prompt_infos = [
    {
        "name": "pre_purchase", 
        "description": "Good for pushing products to customer for selling when customers are in pre-purchase phase.", 
        "prompt_template": pre_purchase_template
    },
    {
        "name": "post_purchase", 
        "description": "Good for answering post-purchase queries to customers in to the point and a crisp manner.", 
        "prompt_template": post_purchase_template
    },
    {
        "name": "angry_customer", 
        "description": "Good for talking polietly to customer with apologizing attitude.", 
        "prompt_template": angry_customer_template
    } 
]

In [ ]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
default_prompt = ChatPromptTemplate.from_template(default_template)
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt strcitly \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json in proper mardown format even if destination is DEFAULT)>>"""

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [ ]:
print(chain.run("where the hell is my order, its been 2 weeks now. I want my money back right now"))

In [ ]:
print(chain.run("Is your product acne clear a good cream? I heard it has lots of side effects"))

In [ ]:
print(chain.run("i want to track my order"))

In [ ]:
print(chain.run("what is bitcoin"))

In [ ]:
print(chain.run("what colour dress are you wearing?"))